In [1]:
# Computational and Visualisation Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import string, re
from collections import OrderedDict
from wordcloud import WordCloud
from itertools import chain, groupby

# Pyspark Packages
from pyspark.sql.functions import col, udf
from pyspark.sql.types import *
from pyspark.ml.feature import CountVectorizer, NGram
from pyspark.ml.clustering import LDA
from pyspark.ml import Pipeline

# NLTK Packages
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data] Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Package wordnet is already up-to-date!

## Helper, Utility and Wrapper Modules to assist in the later part of the notebook

In [3]:
lemmatizer = WordNetLemmatizer()
def lemmatize(token):
  """ Function to return lemmatized token"""
  return lemmatizer.lemmatize(token)

In [4]:
punctuation = set(string.punctuation) 
default_stopwords = stopwords.words('english')
custom_stopwords = ['charliecuntskies ', 'bts', 'imisscath ', 'imisscath', 'mcflyforgermany', 'charliecuntskies', 'delongeday', 'rlly', 'barakatday', 'blah']
stopwords_f = set(default_stopwords + custom_stopwords)
pattern_repl_list = [".", ",", "(", ")", "!", "@", "#"]

replacement_patterns = [
(r"won't", "will not"),
(r"haven't", "have not"),
(r"can't", "cannot"),
(r"i'm", "i am"),
(r"ain't", "am not"),
(r"(\w+)'ll", "$1 will"),
(r"(\w+)n't", "$1 not"),
(r"(\w+)'ve", "$1 have"),
(r"(\w+)'s", "$1 is"),
(r"(\w+)'re", "$1 are"),
(r"(\w+)'d", "$1 would"),
]

def preprocessed_chats (text):
  """ Function to extract processed column over Spark Dataframe """
  text_token = text.lower().split()
  
  text_token_filtered = []
  for token in text_token:
    for pattern in pattern_repl_list: token = token.replace(pattern, '') # Limit patterns from pattern_repl_list
    for pattern in replacement_patterns: token = token.replace(pattern[0], pattern[1]) # Limit patterns from replacement_patterns list
    text_token_filtered.append(token)
  
  text_token_filtered_pre = [word for word in text_token_filtered if len(word) > 4]
  text_sw_removed = [word for word in text_token_filtered if word not in stopwords_f]
  text_punc_removed = [word for word in text_sw_removed if word not in punctuation]
  text_lemmatized = [lemmatize(word) for word in text_punc_removed]
  return text_lemmatized

preprocessing_udf = udf(preprocessed_chats, ArrayType(StringType()))

In [5]:
def plotter(x_vals, y_vals, xlabel, ylabel, title, color=None):
  """ Custom Pyplot Plotter"""
  if color is None:
    color='red'
  fig, ax = plt.subplots()
  ax.plot(x_vals, y_vals, color=color)
  ax.set_title(title)
  ax.set_xlabel(xlabel)
  ax.set_ylabel(ylabel)
  display (fig)

In [6]:
def expanded_frame(summary_df):
  """ Function to return words and termWeights each for a particular cluster"""
  df_list = []
  for index, row in summary_df.toPandas().iterrows():
    words_list = row['words']
    words_weight = row['termWeights']

    for itr in range(len(words_list)):
      expanded_dict = OrderedDict()
      expanded_dict['cluster'] = row['cluster']
      expanded_dict['words'] = words_list[itr]
      expanded_dict['termWeights'] = words_weight[itr]
      df_list.append(expanded_dict)

  new_df = spark.createDataFrame(df_list)
  return (new_df)

In [7]:
def return_cluster_summary(lda_model):
  """ Function to retrieve formatted cluster summary for analysis"""
  cluster_summary_df_list = []
  final_df_column_list = ['cluster', 'words', 'termWeights']
  
  vocab = lda_model.stages[0].vocabulary 
  topics = lda_model.stages[1].describeTopics()
  topics_words = topics.rdd.map(lambda row: row['termIndices']) .map(lambda idx_list: [vocab[idx] for idx in idx_list]).collect()
  
  for idx, topic in enumerate(topics_words):
    cluster_summary_dict = OrderedDict()
    cluster_summary_dict['cluster'] = idx + 1
    cluster_summary_dict['words'] = [word for word in topic]
    cluster_summary_df_list.append(cluster_summary_dict)

  summary_pre_df = spark.createDataFrame(cluster_summary_df_list)
  summary_df = summary_pre_df.join(topics, summary_pre_df.cluster == topics.topic)[final_df_column_list]
  #new_summary_df = expanded_frame(summary_df)[final_df_column_list]
  return summary_df

In [8]:
def get_optimal_topics(topic_range, maxIteration, input_df):
  """ Function to compute optimal topics within a custom input range """
  ll_vals, lp_vals, cluster_no = [], [], []
  cv = CountVectorizer(inputCol='tokenized_sentiments', outputCol='features')
  lda = LDA ()
  lda_pipeline = Pipeline(stages=[cv, lda])
  
  for topic_no in topic_range:
    paramap = {lda.k: topic_no, lda.maxIter:maxIteration, cv.vocabSize:100000, cv.minDF:5.0}
    lda_i_model = lda_pipeline.fit (input_df, paramap)
    lda_i_results = lda_i_model.transform (input_df)
    
    ll_vals.append(lda_i_model.stages[1].logLikelihood(lda_i_results))
    lp_vals.append(lda_i_model.stages[1].logPerplexity(lda_i_results))
    cluster_no.append(topic_no)
  
  return ll_vals, lp_vals, cluster_no

In [9]:
def concat(type):
  """ Function to concat multiple Array columns in a dataframe into a single column"""
  def concat_(*args):
      return list(chain(*args))
  return udf(concat_, ArrayType(type))

#PySpark UDF
concat_string_arrays = concat(StringType())

In [10]:
def get_top_ngrams(input_df, cap):
  """ Function to return top ngrams grouped by cluster"""
  top_ngrams_perCluster = {}
  
  for itr in range(0, 12, 1):
    input_df_sub = input_df.filter("cluster_predictions == %d" % itr)
    
    voc_ngram = list(chain(*input_df_sub[['tokenized_sentiments']].toPandas()['tokenized_sentiments'].tolist()))
    voc_ngram_freq = {key:len(list(group)) for key, group in groupby(voc_ngram)}
    voc_ngram_freq_sorted = OrderedDict(sorted(voc_ngram_freq.items(), key=lambda it: it[1], reverse=True))
    top_ngrams_i =  ' '.join(list(set([key.encode('utf-8') for key,value in voc_ngram_freq_sorted.items()[:3000]]))[:cap])
    
    top_ngrams_perCluster[itr] = top_ngrams_i
  
  return top_ngrams_perCluster

In [11]:
%sh 
##  /FileStore/tables/twitter_sentiment_analysis2-9b5de.zip
#cd /dbfs/FileStore/tables/
##unzip twitter_sentiment_analysis2-9b5de.zip
#mkdir -p /dbfs/mnt/lda/
#cp train.csv /dbfs/mnt/lda/twitter_sentiment.csv

In [12]:
twitter_sentiments = spark.read.csv('/mnt/lda/twitter_sentiment.csv', header=True) # Only taking a limited set
twitter_sentiments = twitter_sentiments[['SentimentText']]
twitter_sentiments = twitter_sentiments.withColumn("tokenized_sentiments", preprocessing_udf("SentimentText"))

In [13]:
twitter_sentiments_cv = CountVectorizer(inputCol='tokenized_sentiments', outputCol='features') # Creating featues column
twitter_sentiments_lda = LDA () # Training a LDA Model
Lda_Pipeline = Pipeline(stages=[twitter_sentiments_cv, twitter_sentiments_lda])

# Parameterized values
paramap = {twitter_sentiments_lda.k: 9, twitter_sentiments_lda.maxIter:50, twitter_sentiments_cv.vocabSize:100000, twitter_sentiments_cv.minDF:5.0}

LdaModel = Lda_Pipeline.fit(twitter_sentiments, paramap)
LdaResults = LdaModel.transform(twitter_sentiments)

cluster_summary_df = return_cluster_summary(LdaModel)
display (cluster_summary_df)

cluster,words,termWeights
1,"List(thank, wish, could, like, got, girl, , love, feel, know)","List(0.019832085774291935, 0.017667615192240544, 0.01278841902211623, 0.011497980713773105, 0.008808296122910276, 0.007938521356095735, 0.007317337164365658, 0.007141093835725717, 0.0063817789091509735, 0.005941255764071033)"
2,"List(said, thought, yeah, thing, exactly, mean, hate, myweakness, follow, second)","List(0.03355007465199728, 0.012330520052984035, 0.012009115449253394, 0.010138482418648, 0.010026986684539243, 0.009850389161256641, 0.009019471426372879, 0.00884479560446169, 0.00802454907682287, 0.007851805946270206)"
3,"List(love, wanna, happy, &, u, get, great, lol, day, still)","List(0.01694661951468619, 0.014224388424454037, 0.012726501889125318, 0.012544694003537237, 0.012131699648805765, 0.01113067164982209, 0.010920503419947622, 0.010556856875636546, 0.008662035576269598, 0.008464833324140148)"
4,"List(know, go, cannot, new, want, like, much, get, that's, lol)","List(0.017885742334392003, 0.016150452459641116, 0.016025094114560663, 0.01256018472245425, 0.009629492534559745, 0.009618481037404585, 0.009583313167791324, 0.009235953388504622, 0.007909604283676142, 0.007491956161727438)"
5,"List(, good, i am, u, like, get, lol, time, thanks, day)","List(0.019199325260617323, 0.01502195282466297, 0.009995819493855867, 0.007161779038940221, 0.006584564544502578, 0.0053619964978811555, 0.005168854911937015, 0.005105750354268841, 0.004981154877219053, 0.004862563332871768)"
6,"List(oh, haha, suck, check, two, okay, love, next, talking, away)","List(0.03831686454509927, 0.01060752592295178, 0.010346307644073011, 0.009513023896561132, 0.009340676232261627, 0.009318685573472941, 0.00887796287217453, 0.0065372890494664005, 0.005621639767919185, 0.005521682373443904)"
7,"List(i am, think, come, sorry, working, u, phone, true, , da)","List(0.015580323108866735, 0.015370044738176623, 0.01534565738529194, 0.012133787055996487, 0.012111761866006493, 0.011917484139487605, 0.011821265005306648, 0.011501689157690505, 0.008620704450890233, 0.008460183144512958)"
8,"List(everyone, hey, follower, get, pay, add, using, day, train, link)","List(0.02875325552308516, 0.027086625903748704, 0.022273227169057587, 0.0162124964050247, 0.008924499094948751, 0.00805277279848437, 0.007297107843811206, 0.007245761787022959, 0.005829878821669545, 0.004821426518368476)"


In [14]:
fig, ((t1,t2), (t3,t4), (t5,t6), (t7,t8)) = plt.subplots(4, 2)
wordcloud = WordCloud(background_color='white')

t1.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 1')[['words']].collect()[0].words).encode('utf-8')))
t2.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 2')[['words']].collect()[0].words).encode('utf-8')))
t3.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 3')[['words']].collect()[0].words).encode('utf-8')))
t4.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 4')[['words']].collect()[0].words).encode('utf-8')))
t5.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 5')[['words']].collect()[0].words).encode('utf-8')))
t6.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 6')[['words']].collect()[0].words).encode('utf-8')))
t7.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 7')[['words']].collect()[0].words).encode('utf-8')))
t8.imshow(wordcloud.generate(' '.join(cluster_summary_df.filter('cluster == 8')[['words']].collect()[0].words).encode('utf-8')))

t1.set_title('Cluster 1')
t2.set_title('Cluster 2')
t3.set_title('Cluster 3')
t4.set_title('Cluster 4')
t5.set_title('Cluster 5')
t6.set_title('Cluster 6')
t7.set_title('Cluster 7')
t8.set_title('Cluster 8')

t1.figure.set_size_inches(20, 20)
t2.figure.set_size_inches(20, 20)
t3.figure.set_size_inches(20, 20)
t4.figure.set_size_inches(20, 20)
t5.figure.set_size_inches(20, 20)
t6.figure.set_size_inches(20, 20)
t7.figure.set_size_inches(20, 20)
t8.figure.set_size_inches(20, 20)

t1.set_axis_off()
t2.set_axis_off()
t3.set_axis_off()
t4.set_axis_off()
t5.set_axis_off()
t6.set_axis_off()
t7.set_axis_off()
t8.set_axis_off()
display (fig)

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2717300798909995> in <module> () 
 2 wordcloud = WordCloud ( background_color = 'white' ) 
 3 
 ----> 4 t1 . imshow ( wordcloud . generate ( ' ' . join ( cluster_summary_df . filter ( 'cluster == 1' ) [ [ 'words' ] ] . collect ( ) [ 0 ] . words ) . encode ( 'utf-8' ) ) ) 
 5 t2 . imshow ( wordcloud . generate ( ' ' . join ( cluster_summary_df . filter ( 'cluster == 2' ) [ [ 'words' ] ] . collect ( ) [ 0 ] . words ) . encode ( 'utf-8' ) ) ) 
 6 t3 . imshow ( wordcloud . generate ( ' ' . join ( cluster_summary_df . filter ( 'cluster == 3' ) [ [ 'words' ] ] . collect ( ) [ 0 ] . words ) . encode ( 'utf-8' ) ) ) 

 /databricks/python/lib/python3.5/site-packages/wordcloud/wordcloud.py in generate (self, text) 
 603 self 
 604 """
 --> 605 return self . generate_from_text ( text ) 
 606 
 607 def _check_generated ( self ) : 

 /databricks/python/lib/python3.5/site-packages/wordcloud/wordcloud.py in generate_from_text (self, text) 
 584 self 
 585 """
 --> 586 words = self . process_text ( text ) 
 587 self . generate_from_frequencies ( words ) 
 588 return self 

 /databricks/python/lib/python3.5/site-packages/wordcloud/wordcloud.py in process_text (self, text) 
 551 regexp = self . regexp if self . regexp is not None else r"\w[\w']+" 
 552 
 --> 553 words = re . findall ( regexp , text , flags ) 
 554 # remove stopwords 
 555 words = [ word for word in words if word . lower ( ) not in stopwords ] 

 /databricks/python/lib/python3.5/re.py in findall (pattern, string, flags) 
 211 
 212 Empty matches are included in the result."""
 --> 213 return _compile ( pattern , flags ) . findall ( string ) 
 214 
 215 def finditer ( pattern , string , flags = 0 ) : 

 TypeError : cannot use a string pattern on a bytes-like object

In [15]:
topic_range = range(6, 15, 2)
ll_vals, lp_vals, cluster_no = get_optimal_topics(topic_range, 50, twitter_sentiments.sample(False, 0.4))

In [16]:
plotter(cluster_no, ll_vals, xlabel='Clusters', ylabel='LogLikelihood Scores', title='Plot of LogLikelihood against number of clusters', color='green')

In [17]:
plotter(cluster_no, lp_vals, xlabel='Clusters', ylabel='LogPerplexity Scores', title='Plot of LogPerplexity against number of clusters', color='blue')

In [18]:
# Parameterized values
paramap = {twitter_sentiments_lda.k: 13, twitter_sentiments_lda.maxIter:50, twitter_sentiments_cv.vocabSize:100000, twitter_sentiments_cv.minDF:6.0}

LdaModel = Lda_Pipeline.fit(twitter_sentiments, paramap)
LdaResults_Best = LdaModel.transform(twitter_sentiments)

cluster_summary_df_Best = return_cluster_summary(LdaModel)
display (cluster_summary_df_Best)

In [19]:
fig, ((t1,t2), (t3,t4), (t5,t6), (t7,t8), (t9, t10), (t11, t12)) = plt.subplots(6, 2)
wordcloud = WordCloud(background_color='white')

t1.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 1')[['words']].collect()[0].words).encode('utf-8')))
t2.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 2')[['words']].collect()[0].words).encode('utf-8')))
t3.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 3')[['words']].collect()[0].words).encode('utf-8')))
t4.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 4')[['words']].collect()[0].words).encode('utf-8')))
t5.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 5')[['words']].collect()[0].words).encode('utf-8')))
t6.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 6')[['words']].collect()[0].words).encode('utf-8')))
t7.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 7')[['words']].collect()[0].words).encode('utf-8')))
t8.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 8')[['words']].collect()[0].words).encode('utf-8')))
t9.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 9')[['words']].collect()[0].words).encode('utf-8')))
t10.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 10')[['words']].collect()[0].words).encode('utf-8')))
t11.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 11')[['words']].collect()[0].words).encode('utf-8')))
t12.imshow(wordcloud.generate(' '.join(cluster_summary_df_Best.filter('cluster == 12')[['words']].collect()[0].words).encode('utf-8')))

t1.set_title('Cluster 1')
t2.set_title('Cluster 2')
t3.set_title('Cluster 3')
t4.set_title('Cluster 4')
t5.set_title('Cluster 5')
t6.set_title('Cluster 6')
t7.set_title('Cluster 7')
t8.set_title('Cluster 8')
t9.set_title('Cluster 9')
t10.set_title('Cluster 10')
t11.set_title('Cluster 11')
t12.set_title('Cluster 12')

t1.figure.set_size_inches(20, 20)
t2.figure.set_size_inches(20, 20)
t3.figure.set_size_inches(20, 20)
t4.figure.set_size_inches(20, 20)
t5.figure.set_size_inches(20, 20)
t6.figure.set_size_inches(20, 20)
t7.figure.set_size_inches(20, 20)
t8.figure.set_size_inches(20, 20)
t9.figure.set_size_inches(20, 20)
t10.figure.set_size_inches(20, 20)
t11.figure.set_size_inches(20, 20)
t12.figure.set_size_inches(20, 20)

t1.set_axis_off()
t2.set_axis_off()
t3.set_axis_off()
t4.set_axis_off()
t5.set_axis_off()
t6.set_axis_off()
t7.set_axis_off()
t8.set_axis_off()
t9.set_axis_off()
t10.set_axis_off()
t11.set_axis_off()
t12.set_axis_off()
display (fig)

In [20]:
ngram_range = range(2, 5, 1)
ngrams = [NGram(n=ngram, inputCol="tokenized_sentiments", outputCol="{0}_grams".format(ngram)) for ngram in ngram_range]
ngrams_pipeline = Pipeline(stages=ngrams)
ngram_df = ngrams_pipeline.fit(LdaResults).transform(LdaResults)

# Merge Ngrams into a single column
ngram_df = ngram_df.select(col('SentimentText'), col('tokenized_sentiments'), col('features'), col('topicDistribution'), concat_string_arrays(col("2_grams"), col("3_grams"), col("4_grams")).alias('ngrams'))

In [21]:
voc_ngram = list(chain(*ngram_df[['ngrams']].toPandas()['ngrams'].tolist()))
voc_ngram_freq = {key:len(list(group)) for key, group in groupby(voc_ngram)}
voc_ngram_freq_sorted = OrderedDict(sorted(voc_ngram_freq.items(), key=lambda it: it[1], reverse=True))
top_30_ngrams =  ' '.join(list(set([key.encode('utf-8') for key,value in voc_ngram_freq_sorted.items()[:3000]]))[:30])

fig, ax = plt.subplots()
wordcloud = WordCloud(background_color='black')
ax.imshow(wordcloud.generate(top_30_ngrams))
ax.set_title('Top 30 NGrams')
ax.figure.set_size_inches(10, 10)
ax.set_axis_off()
display(fig)

In [22]:
from pyspark.ml.clustering import BisectingKMeans

kmeans_inst = BisectingKMeans(featuresCol='features', predictionCol='cluster_predictions', maxIter=50, k=12, minDivisibleClusterSize=5.0, seed=42)
kmeans_mod = kmeans_inst.fit(LdaResults)
LdaResults_kmeans = kmeans_mod.transform(LdaResults)
cluster_centers = kmeans_mod.clusterCenters()

display (LdaResults_kmeans, 100)

In [23]:
costs = []
clusters = []

for cluster_i in range(6, 15, 2):
    kmeans_i = BisectingKMeans(featuresCol='features', predictionCol='cluster_predictions', maxIter=30, k=cluster_i, minDivisibleClusterSize=5.0, seed=43)
    model_i = kmeans_i.fit(LdaResults_kmeans.sample(False, 0.2, seed=42))
    
    # Updating the list
    costs.append(model_i.computeCost(LdaResults_kmeans))
    clusters.append(cluster_i)

# plotter(x_vals, y_vals, xlabel, ylabel, title, color=None)
plotter (clusters, costs, 'Clusters', 'Cost', 'Detecting Optimal Number of clusters using KMeans', color='blue')

In [24]:
top30_ngrams_perCluster = get_top_ngrams(LdaResults_kmeans, 30)
fig, ((t1,t2), (t3,t4), (t5,t6), (t7,t8), (t9, t10), (t11, t12)) = plt.subplots(6, 2)
wordcloud = WordCloud(background_color='white')

t1.imshow(wordcloud.generate(top30_ngrams_perCluster.get(0)))
t2.imshow(wordcloud.generate(top30_ngrams_perCluster.get(1)))
t3.imshow(wordcloud.generate(top30_ngrams_perCluster.get(2)))
t4.imshow(wordcloud.generate(top30_ngrams_perCluster.get(3)))
t5.imshow(wordcloud.generate(top30_ngrams_perCluster.get(4)))
t6.imshow(wordcloud.generate(top30_ngrams_perCluster.get(5)))
t7.imshow(wordcloud.generate(top30_ngrams_perCluster.get(6)))
t8.imshow(wordcloud.generate(top30_ngrams_perCluster.get(7)))
t9.imshow(wordcloud.generate(top30_ngrams_perCluster.get(8)))
t10.imshow(wordcloud.generate(top30_ngrams_perCluster.get(9)))
t11.imshow(wordcloud.generate(top30_ngrams_perCluster.get(10)))
t12.imshow(wordcloud.generate(top30_ngrams_perCluster.get(11)))

t1.set_title('Cluster 1')
t2.set_title('Cluster 2')
t3.set_title('Cluster 3')
t4.set_title('Cluster 4')
t5.set_title('Cluster 5')
t6.set_title('Cluster 6')
t7.set_title('Cluster 7')
t8.set_title('Cluster 8')
t9.set_title('Cluster 9')
t10.set_title('Cluster 10')
t11.set_title('Cluster 11')
t12.set_title('Cluster 12')

t1.figure.set_size_inches(20, 20)
t2.figure.set_size_inches(20, 20)
t3.figure.set_size_inches(20, 20)
t4.figure.set_size_inches(20, 20)
t5.figure.set_size_inches(20, 20)
t6.figure.set_size_inches(20, 20)
t7.figure.set_size_inches(20, 20)
t8.figure.set_size_inches(20, 20)
t9.figure.set_size_inches(20, 20)
t10.figure.set_size_inches(20, 20)
t11.figure.set_size_inches(20, 20)
t12.figure.set_size_inches(20, 20)

t1.set_axis_off()
t2.set_axis_off()
t3.set_axis_off()
t4.set_axis_off()
t5.set_axis_off()
t6.set_axis_off()
t7.set_axis_off()
t8.set_axis_off()
t9.set_axis_off()
t10.set_axis_off()
t11.set_axis_off()
t12.set_axis_off()
display (fig)

In [25]:
LdaResults_kmeans.createOrReplaceTempView('LdaResults_kmeans')
LdaResults_kmeans_summary = spark.sql('''with tbl1 as (select cluster_predictions AS summ_cluster_predictions, count(*) count_total from LdaResults_kmeans group by cluster_predictions) 
                             select tbl1.*, row_number() over (order by count_total desc) as cluster_prediction from tbl1 order by count_total desc''')
LDA_Results_kmeans_summary_expanded = LdaResults_kmeans_summary.join(LdaResults_kmeans, LdaResults_kmeans_summary.summ_cluster_predictions == LdaResults_kmeans.cluster_predictions, how='inner')[['SentimentText', 'tokenized_sentiments', 'count_total', 'cluster_predictions', 'topicDistribution']]
display (LDA_Results_kmeans_summary_expanded)
spark.catalog.dropTempView('LdaResults_kmeans')

In [26]:
display (LDA_Results_kmeans_summary_expanded)

The dataset we used in this notebook came from a specific user's Facebook feed, so, isn't a very great one to start with. However, both, LDA and Bisecting KMeans implementation demonstrated that 12 topics is an optimal number for topic modeling. 
The published notebook is available at - 
https://databricks-prod-cloudfront.cloud.databricks.com/public/4027ec902e239c93eaaa8714f173bcfc/3173713035751393/2689111109152757/2308983777460038/latest.html

Note: Data Source used is those of the web scrapped data publicly available from Twitter API from personal twitter handle, and other twitter handles owned by me.